# Main Notebook

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from pickles for X different models
 -  Load BaseBid Predictions from pickels for X different models
 -  For X models combine the above two steps to build X BidPrices for each impression
 -  Hold auction for each impression one by one
 -  Summary Scores for each of X models 

# Preparation

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer

%matplotlib inline

# Pull Validation Data

In [8]:
path = "C:/Users/Akis/OneDrive/Masters/Web Economics/Data/"
#test_df = pd.read_csv(path+'test.csv')
#train_df = pd.read_csv(path+'train.csv')
validation_df = pd.read_csv(path+'validation.csv')

In [9]:

#print(validation_df.head())
print(validation_df.shape[0])
print(validation_df.loc[validation_df["click"] == 1].shape[0])
avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
print(avgCTR)

303925
202
0.0006646376573167722


# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [16]:
teamAdf = pd.DataFrame(columns=[],index=range(0,validation_df.shape[0],1))
teamAdf["teamA_CTR"]=np.random.randint(0, 10, teamAdf.shape[0])/10
teamBdf = pd.DataFrame(columns=[],index=range(0,validation_df.shape[0],1))
teamBdf["CTR"]=np.random.randint(0, 10, teamAdf.shape[0])/10
#print(teamAdf)
#print(teamBdf)




resultCTR=teamAdf
resultCTR["teamB_CTR"]=teamBdf["CTR"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
print(resultCTR)

        teamA_CTR  teamB_CTR
0             0.3        0.6
1             0.3        0.6
2             0.1        0.9
3             0.6        0.2
4             0.0        0.8
5             0.2        0.0
6             0.5        0.9
7             0.6        0.8
8             0.5        0.4
9             0.0        0.2
10            0.6        0.4
11            0.9        0.1
12            0.5        0.9
13            0.9        0.2
14            0.7        0.5
15            0.7        0.9
16            0.1        0.6
17            0.4        0.0
18            0.0        0.4
19            0.2        0.8
20            0.4        0.5
21            0.6        0.2
22            0.0        0.9
23            0.4        0.3
24            0.1        0.1
25            0.6        0.0
26            0.4        0.4
27            0.6        0.9
28            0.0        0.1
29            0.4        0.4
...           ...        ...
303895        0.6        0.3
303896        0.6        0.3
303897        

# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [17]:
teamABaseBid = pd.DataFrame(columns=[],index=range(0,validation_df.shape[0],1))
teamABaseBid["teamA_BaseBid"]=250
teamBBaseBid = pd.DataFrame(columns=[],index=range(0,validation_df.shape[0],1))
teamBBaseBid["teamB_BaseBid"]=np.random.randint(227, 301, teamBBaseBid.shape[0])
#print(teamABaseBid)
#print(teamBBaseBid)


resultBaseBid=teamABaseBid
resultBaseBid["teamB_BaseBid"]=teamBBaseBid["teamB_BaseBid"]
#result = pd.merge(teamAdf, teamBdf, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
print(resultBaseBid)

        teamA_BaseBid  teamB_BaseBid
0                 250            292
1                 250            298
2                 250            296
3                 250            253
4                 250            274
5                 250            247
6                 250            273
7                 250            262
8                 250            299
9                 250            227
10                250            293
11                250            242
12                250            254
13                250            271
14                250            232
15                250            282
16                250            242
17                250            252
18                250            257
19                250            227
20                250            254
21                250            269
22                250            275
23                250            276
24                250            279
25                250            287
2

# Build Bid Prices for all models

Need a function that will build a new dataframe conatining bids of all teams by combining the previous two dataframes.

In [18]:
resultBids = resultBaseBid.values * ((resultCTR*(138/35000))/avgCTR)
resultBids2 = resultBaseBid.values * (resultCTR/0.5)
#print(resultBids)
print(resultBids2)

        teamA_CTR  teamB_CTR
0           150.0      350.4
1           150.0      357.6
2            50.0      532.8
3           300.0      101.2
4             0.0      438.4
5           100.0        0.0
6           250.0      491.4
7           300.0      419.2
8           250.0      239.2
9             0.0       90.8
10          300.0      234.4
11          450.0       48.4
12          250.0      457.2
13          450.0      108.4
14          350.0      232.0
15          350.0      507.6
16           50.0      290.4
17          200.0        0.0
18            0.0      205.6
19          100.0      363.2
20          200.0      254.0
21          300.0      107.6
22            0.0      495.0
23          200.0      165.6
24           50.0       55.8
25          300.0        0.0
26          200.0      184.0
27          300.0      505.8
28            0.0       56.6
29          200.0      208.0
...           ...        ...
303895      300.0      154.8
303896      300.0      166.2
303897      15

# Hold Auction for each impression one-by-one

Need a function that will step through the impressions one by one checking who won the impression, whilst keeping track of each player's budget.

In [19]:

resultBids2['click']=np.random.randint(0, 1, resultBids2.shape[0])
resultBids2[['teamA_CTR','teamB_CTR']]
#Scorer()

,teamA_CTR,teamB_CTR
0,150.0,350.4
1,150.0,357.6
2,50.0,532.8
3,300.0,101.2
4,0.0,438.4
5,100.0,0.0
6,250.0,491.4
7,300.0,419.2
8,250.0,239.2
9,0.0,90.8


In [23]:
s = Scorer()
df_summary,df = s.set_df(resultBids2, ['teamA_CTR', 'teamB_CTR'],625000,200)

   click  team_name   win
0      0  teamB_CTR  2691
1      0  teamA_CTR  2793


# Scores

Summarise the main metrics for each team.

In [24]:
print(df_summary)

   click  team_name   win
0      0  teamB_CTR  2691
1      0  teamA_CTR  2793
